In [1]:
import pyspark
import os

os.environ["PYSPARK_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'
os.environ["PYSPARK_DRIVER_PYTHON"]='C:\\Users\\201910810\\Anaconda3\\python.exe'

myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
data=[
    "'김하나','English', 100",
    "'김하나','Math', 80",
    "'임하나','English', 70",
    "'임하나','Math', 100",
    "'김갑돌','English', 82.3",
    "'김갑돌','Math', 98.5"
]
marks = spark.sparkContext.parallelize(data)

# 문제 3-1: 이름으로 합계를 구해보자.

In [4]:
marksByName=marks\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))\
    .reduceByKey(lambda x, y:x+y)\
    .collect()

In [5]:
for i in marksByName:
  print (i[0],i[1])

'김하나' 180.0
'임하나' 170.0
'김갑돌' 180.8


In [10]:
a = marks\
    .map(lambda x:x.split(','))\
    .map(lambda x:(x[0],1))\
    .reduceByKey(lambda x, y:x+y)\
    .collect()

for i, j in zip(marksByName, a):
  print (i[0],i[1], j[1])

'김하나' 180.0 2
'임하나' 170.0 2
'김갑돌' 180.8 2


In [9]:
a

[("'김하나'", 2), ("'임하나'", 2), ("'김갑돌'", 2)]

# 문제 3-2: 과목으로 합계를 계산해 보자.

In [7]:
marksBySubject=marks\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[1],float(x[2])))\
    .reduceByKey(lambda x, y:x+y)\
    .collect()

In [8]:
for i in marksBySubject:
  print (i[0],i[1])

'English' 252.3
'Math' 278.5


# 문제 3-3: 이름으로 합계과 개수를 구해보자.

In [11]:
_marksByName2=marks\
    .map(lambda x:x.split(','))\
    .map(lambda x: (x[0],float(x[2])))

In [12]:
marksByName2 = _marksByName2.combineByKey(lambda value: (value,1),
                     lambda x,value: (x[0]+value, x[1]+1),
                     lambda x,y: (x, y))

In [15]:
marksByName2.collect()

[("'김하나'", (180.0, 2)), ("'임하나'", (170.0, 2)), ("'김갑돌'", (180.8, 2))]

In [18]:
marksByName2.map(lambda x:(x[0],x[1][0]/x[1][1])).collect()

[("'김하나'", 90.0), ("'임하나'", 85.0), ("'김갑돌'", 90.4)]

In [36]:
for i in marksByName2.collect():
    for j in i:
        print (j, end=' ')
    print()

'김하나' (180.0, 2) 
'임하나' (170.0, 2) 
'김갑돌' (180.8, 2) 


# 문제 3-4: 이름으로 평균을 계산해 보자.

In [37]:
avgByKey = marksByName2.map(lambda x: (x[0],x[1][0]/x[1][1]))

In [39]:
for i in avgByKey.collect():
    for j in i:
        print (j, end=' ')
    print()

'김하나' 90.0 
'임하나' 85.0 
'김갑돌' 90.4 
